In [5]:
# this code uses the pre-trained T5-base model developed by Google; trained on CNN-DM 
# https://github.com/google-research/text-to-text-transfer-transformer#released-model-checkpoints
# https://huggingface.co/transformers/model_doc/t5.html

import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# # Setting up the device for GPU usage; haven't figured out why gpu doesn't work
from torch import cuda
#device = 'cuda' if cuda.is_available() else 'cpu'
device =  'cpu'

text ="""
(CNN)The city of Tulsa is experiencing a surge in coronavirus cases, a little over 2 weeks after President Donald Trump held a campaign rally in an indoor arena there.
Dr. Bruce Dart, Executive Director of the Tulsa Health Department, said in a press conference on Wednesday there are high numbers being reported this week, with nearly 500 new cases in two days and trends are showing that those numbers will increase.
There had been a 20% decline in new Covid-19 cases the week of June 28 through July 4.
The Tulsa Health Department reported 266 new cases on Wednesday, bringing the total number in the county to 4,571. There are 17,894 cases in Oklahoma and 452 deaths, according to Johns Hopkins University's tally of cases in the United States.
When asked if the cases in Tulsa are going up due to the rally on June 20, Dart said that there were several large events a little over two weeks ago.
"I guess we just connect the dots," Dart said.
In a statement to CNN, Leanne Stephens of the Tulsa Health Department said, "Our epidemiologists and contact tracers are inundated with following up with Tulsa County residents who are confirmed positive as the numbers have been extremely high in recent days. Yesterday, we set a new single day case high and you can see on our website where the trends are moving."
This coronavirus has a lengthy incubation period -- the time between when someone gets infected to when they start showing symptoms (if they get symptoms at all).The incubation period is about three to 14 days, with symptoms typically appearing "within four or five days after exposure," according to Harvard Medical School.
Trump campaign communications director Tim Murtaugh told CNN, "There were literally no health precautions to speak of as thousands looted, rioted, and protested in the streets and the media reported that it did not lead to a rise in coronavirus cases. Meanwhile, the President's rally was 18 days ago, all attendees had their temperature checked, everyone was provided a mask, and there was plenty of hand sanitizer available for all. It's obvious that the media's concern about large gatherings begins and ends with Trump rallies."
Murtaugh provided a link to CNN reporting on research suggesting that Black Lives Matter protests across the country did not lead to a jump in coronavirus cases.
All of Trump's campaign staffers who attended his rally went into quarantine, however, after interacting with several colleagues who later tested positive for the virus. Multiple Secret Service officers were also instructed to self-quarantine after two of their colleagues who were on site for the rally tested positive.
About 6,200 people attended the rally, according to the Tulsa Fire Department.
The problem with Trump&#39;s pandemic-era events
The problem with Trump's pandemic-era events
Tulsa Mayor G.T. Bynum said the rally, along with the large "serious seven" events -- which the state has identified as weddings, religious events and other large-scale gatherings -- informed an executive order that goes into effect tomorrow requiring events with more than 500 people to receive guidance from the health department to evaluate safety plans.
Event organizers and health officials will work on a case-by-case bases to make safe decisions in relation to the event, according to the executive order.
While hospital capacity is fine right now, Bynum said that this week he "finally started to hear some concern, not about where things stand today, but where things could look if we continue on this trajectory unchecked."
Bynum said that ordering masks for employees at bars and restaurants has helped the Tulsa Health Department with their contact tracing by showing them what the leading contributors are to cases in the Tulsa area.
There is no city-wide mandate for face coverings, although Bynum and Dart encourage everyone to wear a mask in public.
"I think that the thing that citizens need to understand is that when we put that kind of mandate in place, we will be putting it there because we had no other choice but to do that to protect their ability to get medical care over the long term of this pandemic," Bynum said when asked why the city hasn't put in a mask mandate.
CNN's Holly Yan, Ryan Nobles and Kaitlan Collins contributed to this story.
"""


preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (979 > 512). Running this sequence through the model will result in indexing errors


original text preprocessed: 
 (CNN)The city of Tulsa is experiencing a surge in coronavirus cases, a little over 2 weeks after President Donald Trump held a campaign rally in an indoor arena there.Dr. Bruce Dart, Executive Director of the Tulsa Health Department, said in a press conference on Wednesday there are high numbers being reported this week, with nearly 500 new cases in two days and trends are showing that those numbers will increase.There had been a 20% decline in new Covid-19 cases the week of June 28 through July 4.The Tulsa Health Department reported 266 new cases on Wednesday, bringing the total number in the county to 4,571. There are 17,894 cases in Oklahoma and 452 deaths, according to Johns Hopkins University's tally of cases in the United States.When asked if the cases in Tulsa are going up due to the rally on June 20, Dart said that there were several large events a little over two weeks ago."I guess we just connect the dots," Dart said.In a statement to CNN, Leanne

In [6]:
# The limit of T5 is that the input during training had a window length of 512 words,as is shown in the 

In [7]:
# a slightly differnt way to use T5

from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)


output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.




Summarized text: 
 the city of Tulsa is experiencing a surge in coronavirus cases. there had been a 20% decline in new cases the week of June 28 through July 4. there are 17,894 cases in Oklahoma and 452 deaths.
